In [1]:
import numpy as np
import random
from math import *
import time

import copy

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
torch.set_default_tensor_type('torch.DoubleTensor')

In [2]:
# defination of activation function
def activation(x):
    return x * torch.sigmoid(x) 

In [3]:
# build ResNet with one blocks
class Net(nn.Module):
    def __init__(self,input_size,width):
        super(Net,self).__init__()
        self.layer_in = nn.Linear(input_size,width)
        self.layer_1 = nn.Linear(width,width)
        self.layer_2 = nn.Linear(width,width)
        self.layer_out = nn.Linear(width,1)
    def forward(self,x):
        output = self.layer_in(x)
        output = output + activation(self.layer_2(activation(self.layer_1(output)))) # residual block 1
        output = self.layer_out(output)
        return output

In [4]:
input_size = 1
width = 4

In [5]:
# exact solution
def u_ex(x):  
    return torch.sin(pi*x)

In [6]:
# f(x)
def f(x):
    return pi**2 * torch.sin(pi*x)

In [7]:
grid_num = 200
x = torch.zeros(grid_num + 1, input_size)
for index in range(grid_num + 1):
    x[index] = index * 1 / grid_num

In [8]:
net = Net(input_size,width)

In [9]:
def model(x):
    return x * (x - 1.0) * net(x)

In [10]:
# loss function to DGM by auto differential
def loss_function(x):
    h = 1 / grid_num
    sum_0 = 0.0
    sum_1 = 0.0
    sum_2 = 0.0
    sum_a = 0.0
    sum_b = 0.0
    for index in range(grid_num):
        x_temp = x[index] + h / 2 
        x_temp.requires_grad = True
        
        grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
            
        sum_1 += ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    for index in range(1, grid_num):
        x_temp = x[index]
        x_temp.requires_grad = True
        
        grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        
        sum_2 += ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    x_temp = x[0]
    x_temp.requires_grad = True

    grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
        
    sum_a = ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    x_temp = x[grid_num]
    x_temp.requires_grad = True
    
    grad_x_temp = torch.autograd.grad(outputs = model(x_temp), inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    grad_grad_x_temp = torch.autograd.grad(outputs = grad_x_temp[0], inputs = x_temp, grad_outputs = torch.ones(model(x_temp).shape), create_graph = True)
    
    sum_b = ((grad_grad_x_temp[0])[0] + f(x_temp)[0])**2
    
    sum_0 = h / 6 * (sum_a + 4 * sum_1 + 2 * sum_2 + sum_b)
    return sum_0

In [15]:
def error_function(x):
    error = 0.0
    for index in range(len(x)):
        x_temp = x[index]
        error += (model(x_temp)[0] - u_ex(x_temp)[0])**2
    return error / len(x)

In [12]:
# load model parameters on cpu
pretrained_dict = torch.load('net_params_DRM.pkl', map_location = 'cpu')
    
# get state_dict
net_state_dict = net.state_dict()

# remove keys that does not belong to net_state_dict
pretrained_dict_1 = {k: v for k, v in pretrained_dict.items() if k in net_state_dict}

# update dict
net_state_dict.update(pretrained_dict_1)

# set new dict back to net
net.load_state_dict(net_state_dict)

<All keys matched successfully>

In [14]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [16]:
epoch = 1000

loss_record = np.zeros(epoch)
error_record = np.zeros(epoch)

time_start = time.time()
for i in range(epoch):
    optimizer.zero_grad()
    loss = loss_function(x)
    loss_record[i] = float(loss)
    error = error_function(x)
    error_record[i] = float(error)
    
    if i % 1 == 0:
        print('[Epoch: {:}/{:}], DGM: {:.2e}, error: {:.2e}'.format(i, epoch, loss.item(), error.item()))
    if i == epoch - 1:
        # save model parameters
        torch.save(net.state_dict(), 'net_params_DRM_to_DGM.pkl')
    
    loss.backward()
    optimizer.step() 
    torch.cuda.empty_cache() # clear memory
    
np.save("loss_of_DRM.npy", loss_record)
np.save("error.npy", error_record)

time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

[Epoch: 0/1000], DGM: 1.04e-01, error: 2.20e-06
[Epoch: 1/1000], DGM: 8.27e-02, error: 4.29e-06
[Epoch: 2/1000], DGM: 6.93e-02, error: 2.03e-05
[Epoch: 3/1000], DGM: 6.24e-02, error: 4.11e-05
[Epoch: 4/1000], DGM: 5.88e-02, error: 5.65e-05
[Epoch: 5/1000], DGM: 5.57e-02, error: 6.11e-05
[Epoch: 6/1000], DGM: 5.21e-02, error: 5.51e-05
[Epoch: 7/1000], DGM: 4.81e-02, error: 4.23e-05
[Epoch: 8/1000], DGM: 4.43e-02, error: 2.70e-05
[Epoch: 9/1000], DGM: 4.15e-02, error: 1.34e-05
[Epoch: 10/1000], DGM: 3.98e-02, error: 4.66e-06
[Epoch: 11/1000], DGM: 3.94e-02, error: 2.82e-06
[Epoch: 12/1000], DGM: 3.98e-02, error: 7.54e-06
[Epoch: 13/1000], DGM: 4.04e-02, error: 1.60e-05
[Epoch: 14/1000], DGM: 4.09e-02, error: 2.38e-05
[Epoch: 15/1000], DGM: 4.09e-02, error: 2.76e-05
[Epoch: 16/1000], DGM: 4.03e-02, error: 2.63e-05
[Epoch: 17/1000], DGM: 3.95e-02, error: 2.10e-05
[Epoch: 18/1000], DGM: 3.86e-02, error: 1.42e-05
[Epoch: 19/1000], DGM: 3.78e-02, error: 8.36e-06
[Epoch: 20/1000], DGM: 3.72e-0